In [9]:
import json

file_name = "cors_arra_00000000002"
default_json_file_path = f"Static_Files\\Learn_Page\\All_Courses_JSON\\{file_name}.json"

# Read the JSON data
with open(default_json_file_path, 'r') as f:
    data = json.load(f)

# Now you can access the data using the appropriate keys
print(data)  # This will print the entire JSON data




{'ID': 'cors_arra_00000000002', 'title': 'Array', 'Course': ['Course_1', 'Course_2', 'Course_3', 'Course_4'], 'Test': ['Test_1', 'Test_2', 'Test_3', 'Test_4']}


In [10]:
for key, value in data.items():
    print(f"{key}: {value}")

ID: cors_arra_00000000002
title: Array
Course: ['Course_1', 'Course_2', 'Course_3', 'Course_4']
Test: ['Test_1', 'Test_2', 'Test_3', 'Test_4']
